In [1]:
import sys
sys.path.append("..")
from sif_src.loader import load_data, save_processed_data
from sif_src.preprocess_data import preprocess_text
from sif_src.utils import load_glove, load_glove_vectors
from sif_src.sif import compute_word_frequencies, compute_sif_weights, compute_sif_embeddings, remove_pc
from pathlib import Path
import pandas as pd
import numpy as np
import pickle

[nltk_data] Downloading package punkt to /Users/yegor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yegor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_pickle("../pickle_backups/marco_train_df2024-06-04T17.38.1717490321.pickle")
valid_df = pd.read_pickle("../pickle_backups/marco_valid_df2024-06-04T17.38.1717490321.pickle")

In [3]:
glove_vectors = load_glove_vectors('../wv/glove.6B.50d.txt')

Loading GloVe Vectors:   0%|          | 0/400000 [00:00<?, ?it/s]

Loading GloVe Vectors: 100%|██████████| 400000/400000 [00:27<00:00, 14782.28it/s]


In [4]:
def extract_passage_texts(row):
    passage_dicts = row['passages']
    passage_texts = [passage_dicts['passage_text'][i] for i in range(len(passage_dicts['passage_text']))]
    return " ".join(passage_texts)

# Apply the function to the DataFrame
train_df['passage_text'] = train_df.apply(extract_passage_texts, axis=1)
valid_df['passage_text'] = valid_df.apply(extract_passage_texts, axis=1)

In [5]:
train_corpus = train_df['query'].tolist() + train_df['passage_text'].tolist()
word_freq = compute_word_frequencies(train_corpus)
sif_weights = compute_sif_weights(word_freq)

In [11]:
valid_df

,answers,passages,query,query_id,query_type,wellFormedAnswers,passage_text
0,"[Approximately $15,000 per year.]","{'is_selected': [1, 0, 0, 0, 0, 0], 'passage_t...",walgreens store sales average,9652,numeric,[],The average Walgreens salary ranges from appro...
1,"[$21,550 per year, The average hourly wage for...","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0], 'pas...",how much do bartenders make,9653,numeric,[],A bartender’s income is comprised mostly of ti...
2,"[A boil, also called a furuncle, is a deep fol...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0], 'pas...",what is a furuncle boil,9654,description,[],"Knowledge center. A boil, also known as a furu..."
3,"[Detect and assess a wide range of disorders, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what can urinalysis detect,9655,description,[],Urinalysis: One way to test for bladder cancer...
4,"[Shigellosis, diseases of the nervous system, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what is vitamin a used for,9656,description,[],Since vitamin A is fat-soluble it is not neede...
...,...,...,...,...,...,...,...
111135,"[It is defined as the quality of being wide, o...","{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",define width,129177,NUMERIC,[],1 The state of being wide. 2 The measurement ...
111136,[Human Resource Management is the term used to...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",forbes definition of human resource management,190655,DESCRIPTION,[],Human Resource Management (HRM) is the term us...
111137,[No Answer Present.],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",how long to wax urethane paint,272679,NUMERIC,[],Cleans up with mineral spirits. Can be recoate...
111138,[No Answer Present.],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",easiest way to check accounts payable entered ...,177416,DESCRIPTION,[],1 Set up an Item called Bad Debt Write-Off. Th...
